- Obtain and preprocess a customer churn dataset.
- Split the data into training and testing sets.
- Implement a feed-forward neural network using PyTorch or TensorFlow.
- Experiment with different network architectures, activation functions, and learning rates.
- Evaluate the model using appropriate metrics, such as accuracy, precision, recall, and F1 score.

In [148]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer

SEED = 53


## Obtain and Preprocess a Customer Churn Dataset

***

### Obtain the datasets from Kaggle

In [149]:
# Read the datasets
member = pd.read_csv("Dataset/members_v3.csv")
isChurn = pd.read_csv("Dataset/train_v2.csv")
transaction = pd.read_csv("Dataset/transactions_v2.csv")
userlog = pd.read_csv("Dataset/user_logs_v2.csv")

# Print the datasets
display(member.head())
display(isChurn.head())
display(transaction.head())
display(userlog.head())

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558


### Aggregation

In [150]:
# Check if aggregation is required
if max(member.msno.value_counts()) > 1:
    print("Dataset \"member\" requires aggregation.") 
if max(isChurn.msno.value_counts()) > 1:
    print("Dataset \"isChurn\" requires aggregation.") 
if max(transaction.msno.value_counts()) > 1:
    print("Dataset \"transaction\" requires aggregation.") 
if max(userlog.msno.value_counts()) > 1:
    print("Dataset \"userlog\" requires aggregation.") 

Dataset "transaction" requires aggregation.
Dataset "userlog" requires aggregation.


#### Dataset `transaction`

In [151]:
# Derive unpaid value
transaction["unpaid"] = (transaction["plan_list_price"] < transaction["actual_amount_paid"]).astype(int)

# Define aggregation functions
transaction_agg_func = {"payment_method_id": ["last", "nunique"],
                        "payment_plan_days": ["last", "nunique"],
                        "plan_list_price": ["last"],
                        "is_auto_renew": ["last"],
                        "transaction_date": ["last", "nunique"],
                        "is_cancel": ["max"],
                        "unpaid" :["max"]}

# Group and aggregate
transaction = transaction.sort_values(by = "transaction_date", ascending = False).groupby(["msno"]).agg(transaction_agg_func).droplevel(0, axis = 1).reset_index()

# Rename columns
transaction.columns = ["msno", 
                       "current_payment_method", 
                       "count_payment_methods", 
                       "current_plan_days", 
                       "count_plan_days", 
                       "current_plan_price", 
                       "is_auto_renew", 
                       "last_transaction_date", 
                       "count_transactions", 
                       "ever_cancel",
                       "ever_unpaid"]

display(transaction.head())

,msno,current_payment_method,count_payment_methods,current_plan_days,count_plan_days,current_plan_price,is_auto_renew,last_transaction_date,count_transactions,ever_cancel,ever_unpaid
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,1,395,1,1599,0,20161023,1,0,0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,1,30,1,99,1,20170315,1,0,0
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,1,30,1,149,1,20170228,2,0,0
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,1,30,1,149,1,20170326,1,0,0
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,1,30,1,149,1,20170315,1,0,0


#### Dataset `userlog`

In [152]:
# Drop unuseful information
userlog.drop("date", axis = 1, inplace = True)

# Derive num_all value (number of songs played on the day)
userlog["num_all"] = userlog["num_25"] + userlog["num_50"] + userlog["num_75"] + userlog["num_985"] + userlog["num_100"]

# Transform numbers into percentage
for col in ["num_25", "num_50", "num_75", "num_985", "num_100"]:
    userlog[col] = userlog[col] / userlog["num_all"]

# Group, aggregate, and rename columns
userlog = userlog.groupby(["msno"]).agg("mean").reset_index().rename(columns={"num_25": "perc_25",
                                                                               "num_50": "perc_50",
                                                                                "num_75": "perc_75",
                                                                                "num_985": "perc_985",
                                                                                "num_100": "perc_100",
                                                                                "num_unq": "daily_song_unq",
                                                                                "num_all": "daily_song_all",
                                                                                "total_secs": "daily_secs"})

display(userlog.head())

,msno,perc_25,perc_50,perc_75,perc_985,perc_100,daily_song_unq,daily_secs,daily_song_all
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0.201770,0.034825,0.017038,0.010869,0.735498,20.384615,4534.900962,22.461538
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.178644,0.084537,0.066644,0.138038,0.532136,28.548387,6210.577161,35.129032
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.070211,0.033383,0.022997,0.033052,0.840357,16.714286,4121.830714,20.250000
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.234491,0.126094,0.089477,0.068030,0.481908,39.428571,7137.931333,46.190476
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0.143804,0.039434,0.045320,0.077964,0.693479,7.931034,4014.939552,23.517241


### Mapping

In [153]:
# Get a list of customers ID with available information in all datasets
msno_list = list(set(member.msno) & set(isChurn.msno) & set(transaction.msno) & set(userlog.msno))

# Merge datasets
df = isChurn.merge(member, left_on = "msno", right_on = "msno", how = "left")
df = df.merge(transaction, left_on = "msno", right_on = "msno", how = "left")
df = df.merge(userlog, left_on = "msno", right_on = "msno", how = "left")

# Keep the ones with available information in all datasets
df = df[df.msno.isin(msno_list)]

### Drop Observations with NA and Abnormal Values

In [154]:
# Drop observations with null values
null_rows = pd.DataFrame(df.isnull().any(axis = 1))
null_rows = null_rows[null_rows[0] == True].index

df.drop(null_rows, inplace = True)
df = df[(df.bd >= 18) & (df.bd <= 100)]

# Print the aggregated dataset
print(df.shape)
display(df.head())

(325695, 25)


,msno,is_churn,city,bd,gender,registered_via,registration_init_time,current_payment_method,count_payment_methods,current_plan_days,...,ever_cancel,ever_unpaid,perc_25,perc_50,perc_75,perc_985,perc_100,daily_song_unq,daily_secs,daily_song_all
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0,36.0,1.0,30.0,...,0.0,0.0,0.000000,0.128788,0.098485,0.000000,0.772727,5.000000,1164.418167,5.333333
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0,17.0,2.0,60.0,...,0.0,0.0,0.467898,0.123159,0.058325,0.044534,0.306084,21.600000,3390.523350,27.750000
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0,41.0,1.0,30.0,...,1.0,0.0,0.013364,0.007416,0.004549,0.004682,0.969990,36.533333,15992.149400,65.733333
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1,9.0,28.0,female,3.0,20140202.0,38.0,1.0,410.0,...,0.0,0.0,0.078042,0.022657,0.026831,0.016920,0.855550,24.451613,9381.670806,37.645161
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1,13.0,21.0,female,9.0,20140212.0,38.0,1.0,30.0,...,0.0,0.0,0.268019,0.071833,0.028834,0.021837,0.609477,31.551724,6176.864000,38.517241


In [155]:
num_col = ["bd", "registration_init_time", "count_payment_methods", "current_plan_days", "count_plan_days", "current_plan_price", "last_transaction_date", "count_transactions",
           "perc_25", "perc_50", "perc_75", "perc_985", "perc_100", "daily_song_unq", "daily_secs", "daily_song_all"]
cat_col = ["city", "gender", "registered_via", "current_payment_method", "is_auto_renew", "ever_cancel", "ever_unpaid"]

assert len(df.columns) == len(num_col) + len(cat_col) + 2 # msno and is_chrun

num_pipeline = make_pipeline(StandardScaler())
cat_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_col),
    ("cat", cat_pipeline, cat_col),
])

processed_df = preprocessing.fit_transform(df)
processed_df = pd.DataFrame(processed_df.toarray(), columns = preprocessing.get_feature_names_out(), index = df.index)

target = df.is_churn

In [156]:
tmp_save = processed_df.copy()
tmp_save["target"] = target

tmp_save.to_csv("Dataset/processed.csv", index = False)

## Split the data into training and testing sets

## Implement a feed-forward neural network using PyTorch

## Experiment with Different Network Architectures, Activation Functions, and Learning Rates